In [4]:
from pyomo.environ import SolverFactory, ConstraintList, ConcreteModel, Var, NonNegativeIntegers, NonNegativeReals, \
    Objective, SolverFactory, assert_optimal_termination
import pandas as pd

In [2]:
# page 94 例1 
# 三个加工厂A1 A2 A3, 生产量7， 4， 9
# 四个销售点B1, B2, B3, B4， 需求3，6，5，6
# 求最优调度

In [2]:
factory = {"A1": 7, "A2": 4, "A3": 9}
market = {"B1": 3, "B2": 6, "B3": 5, "B4": 6}
cost_price = {
    ("A1", "B1"): 3,
    ("A1", "B2"): 11,
    ("A1", "B3"): 3,
    ("A1", "B4"): 10,
    ("A2", "B1"): 1,
    ("A2", "B2"): 9,
    ("A2", "B3"): 2,
    ("A2", "B4"): 8,
    ("A3", "B1"): 7,
    ("A3", "B2"): 4,
    ("A3", "B3"): 10,
    ("A3", "B4"): 5,
}
df = pd.DataFrame(0, index=factory, columns=market)
for (x, y), v in cost_price.items():
    df.loc[x, y] = v
df

,B1,B2,B3,B4
A1,3,11,3,10
A2,1,9,2,8
A3,7,4,10,5


In [3]:
# 建模编码
model = ConcreteModel()

model.x = Var(factory, market, domain=NonNegativeReals)

model.cost = Objective(expr=sum(model.x[x, y] * cost_price[(x, y)] for x in factory for y in market),
                       sense=minimize)

model.supply = ConstraintList()
for f, v in factory.items():
    model.supply.add(expr=sum(model.x[f, m] for m in market) == v)
model.demand = ConstraintList()
for m, v in market.items():
    model.demand.add(expr=sum(model.x[f, m] for f in factory) == v)


In [5]:
# 丢进求解器
SolverFactory('glpk', executable='/usr/local/bin/glpsol').solve(model)

In [12]:
print(f"min cost = {model.cost()}")
df = pd.DataFrame(0, index=factory, columns=market)
for x, y in cost_price:
    df.loc[x, y] = model.x[x, y]()
df


==Solution==
min cost = 85


,B1,B2,B3,B4
A1,0,0,5,2
A2,3,0,0,1
A3,0,6,0,3
